In [84]:
# Constants
ALPH = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
PROBABILITIES = [0.0792, 0.0171, 0.0433, 0.0174, 0.0305, 0.0841, 0.0105, 0.0175,
                 0.0683, 0.0112, 0.0336, 0.0500, 0.0326, 0.0672, 0.1108, 0.0281,
                 0.0445, 0.0533, 0.0618, 0.0280, 0.0019, 0.0089, 0.0036, 0.0147,
                 0.0081, 0.0037, 0.0002, 0.0196, 0.0192, 0.0038, 0.0061, 0.0213]
ALPH_SIZE = len(ALPH)
EXPECTED_I = sum([p**2 for p in PROBABILITIES])
I0 = 1.0 / ALPH_SIZE
FILE_NAME = "text.txt"

for i in range(ALPH_SIZE):
    print(f"{ALPH[i]} : {PROBABILITIES[i]}")

sum(PROBABILITIES)

а : 0.0792
б : 0.0171
в : 0.0433
г : 0.0174
д : 0.0305
е : 0.0841
ж : 0.0105
з : 0.0175
и : 0.0683
й : 0.0112
к : 0.0336
л : 0.05
м : 0.0326
н : 0.0672
о : 0.1108
п : 0.0281
р : 0.0445
с : 0.0533
т : 0.0618
у : 0.028
ф : 0.0019
х : 0.0089
ц : 0.0036
ч : 0.0147
ш : 0.0081
щ : 0.0037
ъ : 0.0002
ы : 0.0196
ь : 0.0192
э : 0.0038
ю : 0.0061
я : 0.0213


1.0001000000000002

In [85]:
def transform_symbol(_c):
    if 'а' <= _c and _c <= 'я':
        return _c
    elif _c <= 'Я' and _c >= 'А':
        return _c.lower()
    elif _c == 'Ё' or _c == 'ё':
        return 'е'
    else:
        return ""
    
def preprocess_text(_text):
    text_formatted = ""
    # Change symbols according to requirements
    for c in _text:
        text_formatted += transform_symbol(c)

    # Remove consequtive spaces
    text_formatted = ' '.join(text_formatted.split())
    return text_formatted

def read_text(filename):
    f = open(filename, "r", encoding='utf-8')
    text = f.read()
    f.close()
    return preprocess_text(text)

In [86]:
def affinity_index(text: list) -> int:
    sum = 0
    for c in ALPH:
        occurances = text.count(c)
        sum += occurances * (occurances - 1)
    return sum / (len(text) * (len(text) - 1))

def divide_into_blocks(text: list, r: int):
    res = []
    for i in range(r):
        block = text[i::r]
        res.append(block)
    # print(res)
    return res

# Small optimisation -> works for remotely small lenghts
def find_periods(length: int):
    periods = []
    for d in range(2, length): # 
        if length % d == 0:
            periods.append(d)
    return periods

def af_idx_for_blocks(text: list, r: int):
    idxs = []
    blocks = divide_into_blocks(text, r)
    for i in range(r):
        idxs.append(affinity_index(blocks[i]))
    return idxs

def summary_af_idx_for_diff_r(text: list):
    idx_table = {}
    periods = find_periods(len(text))
    print(periods)
    for r in periods:
        idx_table[r] = sum(af_idx_for_blocks(text, r))
    return idx_table

In [88]:
for (k, v) in summary_af_idx_for_diff_r(read_text(FILE_NAME)).items():
    print(f"r = {k}\nExpected: {EXPECTED_I * k}\nReal:{v}")

[2, 4, 8, 16, 32, 229, 458, 916, 1832, 3664]
r = 2
Expected: 0.11060025999999999
Real:0.07253641096435856
r = 4
Expected: 0.22120051999999998
Real:0.14547256253890423
r = 8
Expected: 0.44240103999999997
Real:0.2913809148829551
r = 16
Expected: 0.8848020799999999
Real:0.5819135619619122
r = 32
Expected: 1.7696041599999999
Real:1.1681605761127711
r = 229
Expected: 12.66372977
Real:7.872983870967742
r = 458
Expected: 25.32745954
Real:16.366666666666692
r = 916
Expected: 50.65491908
Real:30.607142857142843
r = 1832
Expected: 101.30983816
Real:59.49999999999972
r = 3664
Expected: 202.61967632
Real:129.0
